In [2]:
import investpy as ip
import yahooquery as yq
from pandas_datareader import data as pdr
import pandas as pd
from tqdm import trange
import yfinance as yf


ModuleNotFoundError: No module named 'investpy'

In [6]:
df = pd.read_excel('./datas/MSCIeurope.xlsx')
df.head()

,Ticker,Ticker Yahoo,Nom,Secteur,Equity class,Market Share,Weights,Valeur notionnelle,Actions,ISIN,Currency
0,NESN,NSRGY,NESTLE SA,Biens de consommation de base,Actions,"EUR 48 274 991,14",4.67,48274991.14,420838,CH0038863350,CHF
1,ASML,ASML,ASML HOLDING NV,Technologie de l'informatioon,Actions,"EUR 34 906 428,20",3.38,34906428.20,60802,NL0010273215,EUR
2,ROG,ROG.SW,ROCHE HOLDING PAR AG,Santé,Actions,"EUR 33 188 774,68",3.21,33188774.68,104989,CH0012032048,CHF
3,MC,MC.PA,LVMH,Biens de consommation cycliques,Actions,"EUR 29 028 191,40",2.81,29028191.40,41457,FR0000121014,EUR
4,NOVOB,NVO,NOVO NORDISK CLASS B,Santé,Actions,"EUR 28 242 417,87",2.73,28242417.87,247375,DK0060534915,DKK


## get yahoo ticker

In [2]:
def find_ticker(isin):
    try:
        company_name = ip.stocks.search_stocks(by='isin', value=str(isin))
        company_name = company_name["name"][0].split(' ')[0]
        symbol = yq.search(company_name)["quotes"][0]["symbol"]
    except:
        symbol = None
    return symbol

In [ ]:
for i in range(len(df)):
    df.loc[i, "Ticker Yahoo"] = find_ticker(df.loc[i, "ISIN"])
df = df[['Ticker', 'Ticker Yahoo', 'Nom', 'Secteur', 'Equity class', 'Market Share', 'Weights',
       'Valeur notionnelle', 'Actions', 'ISIN', 'Currency', ]] 

In [10]:
# total weight
print(df["Weights"].sum())
# lost weights
print(df.loc[(df['Ticker Yahoo'].isna()), 'Weights'].sum())

99.98999999999998
6.82


### get ESG scores

In [11]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = range(len(df)))
# trange = tqdm(range(len(df)))

for i in trange(len(df)):
    ticker = df.loc[i, 'Ticker Yahoo']
    try:
        sus = yf.Ticker(ticker).sustainability
        scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
        esg_scores.loc[i] = [ticker, scores[0], scores[1], scores[2], scores[3]]
    except:
        pass
esg_scores


100%|██████████| 357/357 [09:28<00:00,  1.59s/it]


,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,ROG.SW,2.48,11.54,9.52,23.53
3,MC.PA,0.76,5.93,5.67,12.37
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
352,NaN,NaN,NaN,NaN,NaN
353,NaN,NaN,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,NaN
355,NaN,NaN,NaN,NaN,NaN


In [15]:
# count missing esg scores
print(esg_scores.isna().sum())

Ticker Yahoo         175
Environment Score    175
Social Score         175
Governance Score     175
Total Score          175
dtype: int64


In [12]:
# concat the two dataframes
df = pd.concat([df, esg_scores], axis=1)

In [14]:
# save the dataframe
df.to_excel('./datas/MSCIeurope.xlsx')

### create the datasets with the price for all ticker

In [6]:
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
prices = None
# trange = tqdm(range(len(df)))
for i in trange(len(df)):
    ticker = df.loc[i, 'Ticker Yahoo']
    try:
        data = yf.download(ticker)
        prices = data.history(period="max")
